In [1]:
import pandas as pd
import json
import os
import sys

notebook_dir = os.getcwd()  
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

from Features_extraction.feature_extraction_class import FeatureExtraction
from Data_processing.images_processing_class import ImagesProcessing
from utils import best_preprocessing

In [2]:
data_normal_path = "../Data/normal/"
data_pothole_path = "../Data/potholes/"

image_process = ImagesProcessing(folder_normal=data_normal_path, folder_potholes=data_pothole_path, img_size=(256, 256))
images = image_process.images
Y = image_process.labels
df_Y = pd.DataFrame(Y)

In [3]:
env_feature_extraction_unprocessed_images = FeatureExtraction(image_process)

In [ ]:
# TODO: Change this filter with the good association method-filter find in articles

filters = {
    "SIFT": ["bilateral", 20],  
    "ORB": ["bilateral", 20], 
    "Harris": ["bilateral", 20], 
    "EDGE": ["bilateral", 20], 
    "LBP": ["gaussian", 20], 
    "Gabor": ["bilateral", 20],
    "otsu": ["gaussian", 20],
    "adaptive": ["bilateral", 20],
    "HOG": ["gaussian", 3],
}

best_configs = {}

for method_name, params in filters.items(): 
    filter_name, n_iter = params
    best_configs[method_name] = []
    best_config, all_results = best_preprocessing(
        image_process,
        getattr(env_feature_extraction_unprocessed_images, f"method_{method_name}"),
        filter_name,
        method_name,
        n_iter=n_iter,
        fixed_histogram_method="clahe" if method_name == "SIFT" else None  # ✅ On force CLAHE ici !
    )
    best_configs[method_name].append(best_config)


Extracting Structural Features using HOG
Extracting Structural Features using HOG
Extracting Structural Features using HOG
Best preprocessing configuration for HOG: {'filter': 'gaussian', 'filter_params': ((3, 3), 0), 'histogram': 'none', 'normalize': True, 'f1-score': 0.8216393442622951}


In [8]:
with open("../best_configs_processing.json", "w") as f:
    json.dump(best_configs, f, indent=4)